# PredPatt Tutorial
This is a self-contained tutorial for PredPatt. You'll know how to use PredPatt in your code after going through this short tutorial.

Let's import PredPatt first.

In [1]:
from predpatt import PredPatt

## Input format
PredPatt is built atop [Universal Dependency](http://universaldependencies.org) (UD). Currently, PredPatt takes two kinds of input formats: CoNLL and [Concrete](https://pypi.python.org/pypi/concrete). 

### 1. Load from CoNLL format
The simplest format is CoNLL. Here's an example of input in CoNLL format:

In [2]:
conll_example = """
1	You	you	PRON	PRP	Case=Nom|Person=2|PronType=Prs	2	nsubj	_	_
2	wonder	wonder	VERB	VBP	Mood=Ind|Tense=Pres|VerbForm=Fin	0	root	_	_
3	if	if	SCONJ	IN	_	6	mark	_	_
4	he	he	PRON	PRP	Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs	6	nsubj	_	_
5	was	be	AUX	VBD	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	6	aux	_	_
6	manipulating	manipulate	VERB	VBG	Tense=Pres|VerbForm=Part	2	advcl	_	_
7	the	the	DET	DT	Definite=Def|PronType=Art	8	det	_	_
8	market	market	NOUN	NN	Number=Sing	6	dobj	_	_
9	with	with	ADP	IN	_	12	case	_	_
10	his	he	PRON	PRP$	Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs	12	nmod:poss	_	_
11	bombing	bombing	NOUN	NN	Number=Sing	12	compound	_	_
12	targets	target	NOUN	NNS	Number=Plur	6	nmod	_	SpaceAfter=No
13	.	.	PUNCT	.	_	2	punct	_	_
"""

Next, let's load this example using utils provided by PredPatt.

In [3]:
from predpatt import load_conllu

"`load_conllu`" will create a generator to load multiple CoNLL examples (each of them is separated by an empty line). 
Here we just keep the first example:

In [4]:
conll_example = [ud_parse for sent_id, ud_parse in load_conllu(conll_example)][0]

Let's see what this loaded example looks like:

In [5]:
print conll_example.pprint(K=3)  # K is the number of columns

nsubj(You/0, wonder/1)        root(wonder/1, ROOT/-1)           mark(if/2, manipulating/5)
nsubj(he/3, manipulating/5)   aux(was/4, manipulating/5)        advcl(manipulating/5, wonder/1)
det(the/6, market/7)          dobj(market/7, manipulating/5)    case(with/8, targets/11)
nmod:poss(his/9, targets/11)  compound(bombing/10, targets/11)  nmod(targets/11, manipulating/5)
punct(./12, wonder/1)


### 2. Load from Concrete format
Loading from data in Concrete format is also handy. Take the concrete file in the test directory as an example.

In [6]:
from predpatt import load_comm
fpath = "./test/data.100.fine.all.ud.comm"
concrete_example = [ud_parse for sent_id, ud_parse in load_comm(fpath)][0]

The concrete example looks like:

In [7]:
print concrete_example.pprint(K=3)

compound(Pierre/0, Vinken/1)  nsubj(Vinken/1, join/8)             punct(,/2, Vinken/1)
nummod(61/3, years/4)         nmod:npmod(years/4, old/5)          amod(old/5, Vinken/1)
punct(,/6, Vinken/1)          aux(will/7, join/8)                 root(join/8, ROOT/-1)
det(the/9, board/10)          dobj(board/10, join/8)              case(as/11, director/14)
det(a/12, director/14)        amod(nonexecutive/13, director/14)  nmod(director/14, join/8)
nmod:tmod(Nov./15, join/8)    nummod(29/16, Nov./15)              punct(./17, join/8)


## Play with PredPatt
With the loaded example, we can easily get a PredPatt object as below:

In [8]:
ppatt = PredPatt(concrete_example)

PredPatt provides a straightforward way to show the predicate-argument content. You can enable/disable "`color`" and "`track_rule`" to generate the output that suits your needs.

In [9]:
print " ".join([token.text for token in ppatt.tokens])
print ppatt.pprint(color=True)

Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .
	?a will join ?b as ?c ?d	[join-root]
		?a: Pierre Vinken , 61 years old	[Vinken-nsubj]
		?b: the board	[board-dobj]
		?c: a nonexecutive director	[director-nmod]
		?d: Nov. 29	[Nov.-nmod:tmod]


In practice, iterating the extracted predicates is quite easy. Let's iteratively print out the predicate and its arguments.

In [10]:
print " ".join([token.text for token in ppatt.tokens])
for predicate in ppatt.instances:
    print "\t%s [%s-%s-%d]" % (" ".join(token.text for token in predicate.tokens),
                            predicate.root.text, predicate.root.gov_rel, predicate.root.position)
    for argument in predicate.arguments:
        print "\t\t%s [%s-%s-%d]" % (" ".join(token.text for token in argument.tokens),
                                     argument.root.text, argument.root.gov_rel, argument.root.position)

Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .
	will join as [join-root-8]
		Pierre Vinken , 61 years old [Vinken-nsubj-1]
		the board [board-dobj-10]
		a nonexecutive director [director-nmod-14]
		Nov. 29 [Nov.-nmod:tmod-15]


### Configurable options
PredPatt can be configured to tackle various syntactic phenomena. To do so, you can pass the configurable options to PredPatt.

In [11]:
from predpatt import PredPattOpts
from predpatt.util.ud import dep_v1, dep_v2
resolve_relcl = True  # relative clauses
resolve_appos = True  # appositional modifiers
resolve_amod = True   # adjectival modifiers
resolve_conj = True   # conjuction
resolve_poss = True   # possessives
ud = dep_v1.VERSION   # the version of UD
opts = PredPattOpts(
        resolve_relcl=resolve_relcl,
        resolve_appos=resolve_appos,
        resolve_amod=resolve_amod,
        resolve_conj=resolve_conj,
        resolve_poss=resolve_poss,
        ud=ud
        )

ppatt = PredPatt(concrete_example, opts=opts)
print " ".join([token.text for token in ppatt.tokens])
print ppatt.pprint(color=True)

Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .
	?a is/are 61 years old	[old-amod]
		?a: Pierre Vinken	[Vinken-nsubj]
	?a will join ?b as ?c ?d	[join-root]
		?a: Pierre Vinken , 61 years old	[Vinken-nsubj]
		?b: the board	[board-dobj]
		?c: a nonexecutive director	[director-nmod]
		?d: Nov. 29	[Nov.-nmod:tmod]
	?a is/are nonexecutive	[nonexecutive-amod]
		?a: a director	[director-nmod]
